In [1]:
import os
from pathlib import Path
from PIL import Image
from typing import Any, Literal

from prettytable import PrettyTable
import skimage as ski
import lightning as L
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchmetrics as tm
from torchvision.models import get_model, get_weight
from torchvision.datasets import DatasetFolder

In [4]:
torch.save(get_model("ResNet18"), "./models_for_plots/resnet18.pt")

# Setup

In [2]:
DATASET_PATHS = {
    'fog-detection': Path('./datasets/fog-detection-dataset-prepared'),
    'fog-or-smog': Path('./datasets/fog-or-smog-detection-dataset-prepared'),
    'foggy-cityscapes': Path('./datasets/foggy-cityscapes-image-dataset-prepared')
}

In [3]:
def load_image(path: str):
    img = ski.io.imread(path)
    if img.ndim == 2:  # Handle grayscale
        img = ski.color.gray2rgb(img)
    if img.shape[-1] == 4:  # Handle RGBA
        img = ski.color.rgba2rgb(img)
    img = ski.util.img_as_ubyte(img)
    img = img.squeeze()
    
    return Image.fromarray(img)


def get_dataloader(
    path: Path | str,
    transform: Any | None = None,
    batch_size: int = 32,
    shuffle: bool = False,
    num_workers: int = 0,
    pin_memory: bool = True,
):
    return DataLoader(
        dataset=DatasetFolder(
            root=path,
            loader=load_image,
            extensions=[".jpg", ".png", ".jpeg"],
            transform=transform,
        ),
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        persistent_workers=True if num_workers > 0 else False,
    )

In [4]:
def recursive_getattr(obj: object, name: str):
    names = name.split(".")
    for name in names:
        obj = getattr(obj, name)
    return obj

def recursive_setattr(obj: object, value: Any, name: str):
    names = name.split(".")
    for name in names[:-1]:
        obj = getattr(obj, name)
    setattr(obj, names[-1], value)

In [5]:
class CNNClassifier(L.LightningModule):
    def __init__(
        self,
        model_name: str,
        num_classes: int,
        loss: nn.Module | None = None,
        learning_rate: float= 1e-4,
        optimizer_name: Literal['adam', 'adamw'] = 'adam',
        weights: str | bool = False,
    ):
        super().__init__()
        self.save_hyperparameters()

        self.learning_rate = learning_rate
        self.optimizer_name = optimizer_name

        self.model_name = model_name
        self.num_classes = num_classes
        self.weights = weights
        if weights == True:  # noqa: E712
            self.weights = f"{self.model_name}_Weights.DEFAULT"
        self.model = self._get_model()

        if loss is not None:
            self.criterion= loss
        else:
            self.criterion = nn.CrossEntropyLoss()

        task = "multiclass" if num_classes > 2 else "binary"
        
        self.train_metrics = tm.MetricCollection({
            "accuracy": tm.classification.Accuracy(task=task, num_classes=num_classes),
            "f1": tm.classification.F1Score(task=task, num_classes=num_classes),
            "precision": tm.classification.Precision(task=task, num_classes=num_classes),
            "recall": tm.classification.Recall(task=task, num_classes=num_classes),
        }, prefix="train_")
        self.validation_metrics = self.train_metrics.clone(prefix="val_")
        self.test_metrics = self.train_metrics.clone(prefix="test_")

    def _get_model(self):
        model = get_model(self.model_name, weights=self.weights)

        classifier_path = None
        classifier_in_features = None
        if hasattr(model, 'fc') and isinstance(model.fc, nn.Linear):
            classifier_path = "fc"
            classifier_in_features = model.fc.in_features
        elif hasattr(model, 'classifier'):
            if isinstance(model.classifier, nn.Linear):
                classifier_path = 'classifier'
                classifier_in_features = model.classifier.in_features
            elif isinstance(model.classifier, nn.Sequential):
                for name, module in reversed(list(model.classifier.named_children())):
                    if isinstance(module, nn.Linear):
                        classifier_path = f"classifier.{name}"
                        classifier_in_features = module.in_features
                        break
        elif hasattr(model, 'heads') and hasattr(model.heads, 'head') and isinstance(model.heads.head, nn.Linear):
            classifier_path = 'heads.head'
            classifier_in_features = model.heads.head.in_features

        
        recursive_setattr(
            model, nn.Linear(classifier_in_features, self.num_classes), classifier_path
        )

        return model
    
    def forward(self, x):
        return self.model(x)
    
    def _common_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = self.criterion(logits, labels)
        preds = torch.argmax(logits, dim=1)
        return loss, preds, labels
    
    def training_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)

        self.log("train_loss", loss)
        self.log_dict(self.train_metrics(preds, labels))

        return loss
    
    def on_train_epoch_end(self):
        self.train_metrics.reset()

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)
        self.validation_metrics.update(preds, labels)
        self.log("val_loss", loss)
        return loss
    
    def on_validation_epoch_end(self):
        self.log_dict(self.validation_metrics.compute())
        self.validation_metrics.reset()
    

    def test_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)
        self.test_metrics.update(preds, labels)
        return loss
    
    def on_test_epoch_end(self):
        self.log_dict(self.test_metrics.compute())
        self.test_metrics.reset()

    def configure_optimizers(self):
        if self.optimizer_name == 'adam':
            optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        elif self.optimizer_name == 'adamw':
            optimizer = optim.AdamW(self.parameters(), lr=self.learning_rate)
        else:
            raise ValueError(f"Unsupported optimizer: {self.optimizer_name}.")
        return optimizer

# Training

## ResNet18

In [7]:
SAVE_DIR = Path('runs/classify/CNN')
MODEL_NAME = "ResNet18"
DATASET = 'fog-detection'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=f"{MODEL_NAME}_Weights.DEFAULT",
)

_transform = get_weight(model.weights).transforms()
trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=_transform,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=_transform,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=_transform))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN/ResNet18-fog-detection/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | ResNet           | 11.2 M | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | train
3 | validation_metrics | MetricCollection | 0      | train
4 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB

Epoch 6: 100%|██████████| 12/12 [00:05<00:00,  2.07it/s, v_num=1]          


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:03<00:00,  1.03it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │            1.0            │
│          test_f1          │            1.0            │
│      test_precision       │            1.0            │
│        test_recall        │            1.0            │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.868534505367279     │
│          test_f1          │    0.8584686517715454     │
│      test_precision       │    0.9685863852500916     │
│        test_recall        │    0.7708333134651184     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:12<00:00,  0.55it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.631313145160675     │
│          test_f1          │    0.4592592716217041     │
│      test_precision       │    0.8611111044883728     │
│        test_recall        │    0.31313130259513855    │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,1.0,1.0,1.0,1.0
fog-or-smog,0.8685,0.8585,0.9686,0.7708
foggy-cityscapes,0.6313,0.4593,0.8611,0.3131


In [8]:
SAVE_DIR = Path('runs/classify/CNN')
MODEL_NAME = "ResNet18"
DATASET = 'fog-or-smog'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=f"{MODEL_NAME}_Weights.DEFAULT",
)

_transform = get_weight(model.weights).transforms()
trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=_transform,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=_transform,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=_transform))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN/ResNet18-fog-or-smog/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | ResNet           | 11.2 M | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | train
3 | validation_metrics | MetricCollection | 0      | train
4 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 52/52 [00:08<00:00,  5.96it/s, v_num=1]          

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 52/52 [00:08<00:00,  5.95it/s, v_num=1]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.982758641242981     │
│          test_f1          │    0.9831932783126831     │
│      test_precision       │     0.991525411605835     │
│        test_recall        │    0.9750000238418579     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:15<00:00,  0.47it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.560606062412262     │
│          test_f1          │    0.6836363673210144     │
│      test_precision       │    0.5340909361839294     │
│        test_recall        │    0.9494949579238892     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.9434,0.9516,1.0,0.9077
fog-or-smog,0.9828,0.9832,0.9915,0.975
foggy-cityscapes,0.5606,0.6836,0.5341,0.9495


In [9]:
SAVE_DIR = Path('runs/classify/CNN')
MODEL_NAME = "ResNet18"
DATASET = 'foggy-cityscapes'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=f"{MODEL_NAME}_Weights.DEFAULT",
)

_transform = get_weight(model.weights).transforms()
trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=_transform,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=_transform,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=_transform))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN/ResNet18-foggy-cityscapes/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | ResNet           | 11.2 M | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | train
3 | validation_metrics | MetricCollection | 0      | train
4 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size 

Epoch 7: 100%|██████████| 22/22 [00:17<00:00,  1.26it/s, v_num=1]          


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:03<00:00,  1.28it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7735849022865295     │
│          test_f1          │    0.7931034564971924     │
│      test_precision       │    0.9019607901573181     │
│        test_recall        │    0.7076923251152039     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8448275923728943     │
│          test_f1          │    0.8620689511299133     │
│      test_precision       │    0.7978723645210266     │
│        test_recall        │          0.9375           │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:13<00:00,  0.50it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.939393937587738     │
│          test_f1          │    0.9405940771102905     │
│      test_precision       │    0.9223300814628601     │
│        test_recall        │    0.9595959782600403     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.7736,0.7931,0.902,0.7077
fog-or-smog,0.8448,0.8621,0.7979,0.9375
foggy-cityscapes,0.9394,0.9406,0.9223,0.9596


## MobileNet V3 small

In [6]:
SAVE_DIR = Path('runs/classify/CNN')
MODEL_NAME = "MobileNet_V3_Small"
DATASET = 'fog-detection'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=f"{MODEL_NAME}_Weights.DEFAULT",
)

_transform = get_weight(model.weights).transforms()
trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=_transform,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=_transform,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=_transform))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN/MobileNet_V3_Small-fog-detection/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | MobileNetV3      | 1.5 M  | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | train
3 | validation_metrics | MetricCollection | 0      | train
4 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.080     Total estimated model param

Epoch 9: 100%|██████████| 12/12 [00:05<00:00,  2.06it/s, v_num=1]          

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 12/12 [00:05<00:00,  2.06it/s, v_num=1]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8340517282485962     │
│          test_f1          │    0.8587155938148499     │
│      test_precision       │    0.7672131061553955     │
│        test_recall        │    0.9750000238418579     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:14<00:00,  0.48it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5303030014038086     │
│          test_f1          │    0.6782007217407227     │
│      test_precision       │    0.5157894492149353     │
│        test_recall        │    0.9898989796638489     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.934,0.9489,0.9028,1.0
fog-or-smog,0.8341,0.8587,0.7672,0.975
foggy-cityscapes,0.5303,0.6782,0.5158,0.9899


In [7]:
SAVE_DIR = Path('runs/classify/CNN')
MODEL_NAME = "MobileNet_V3_Small"
DATASET = 'fog-or-smog'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=f"{MODEL_NAME}_Weights.DEFAULT",
)

_transform = get_weight(model.weights).transforms()
trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=_transform,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=_transform,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=_transform))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN/MobileNet_V3_Small-fog-or-smog/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | MobileNetV3      | 1.5 M  | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | train
3 | validation_metrics | MetricCollection | 0      | train
4 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.080     Total estimated model params 

Epoch 9: 100%|██████████| 52/52 [00:08<00:00,  6.02it/s, v_num=1]          

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 52/52 [00:08<00:00,  5.92it/s, v_num=1]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9849137663841248     │
│          test_f1          │    0.9853861927986145     │
│      test_precision       │    0.9874476790428162     │
│        test_recall        │    0.9833333492279053     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:14<00:00,  0.47it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5101010203361511     │
│          test_f1          │    0.6689419746398926     │
│      test_precision       │    0.5051546096801758     │
│        test_recall        │    0.9898989796638489     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.9057,0.9194,0.9661,0.8769
fog-or-smog,0.9849,0.9854,0.9874,0.9833
foggy-cityscapes,0.5101,0.6689,0.5052,0.9899


In [8]:
SAVE_DIR = Path('runs/classify/CNN')
MODEL_NAME = "MobileNet_V3_Small"
DATASET = 'foggy-cityscapes'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=f"{MODEL_NAME}_Weights.DEFAULT",
)

_transform = get_weight(model.weights).transforms()
trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=_transform,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=_transform,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=_transform))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN/MobileNet_V3_Small-foggy-cityscapes/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | MobileNetV3      | 1.5 M  | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | train
3 | validation_metrics | MetricCollection | 0      | train
4 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.080     Total estimated model pa

Epoch 9: 100%|██████████| 22/22 [00:17<00:00,  1.24it/s, v_num=1]          

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 22/22 [00:17<00:00,  1.24it/s, v_num=1]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.704741358757019     │
│          test_f1          │     0.777235746383667     │
│      test_precision       │     0.637333333492279     │
│        test_recall        │    0.9958333373069763     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:13<00:00,  0.50it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8282828330993652     │
│          test_f1          │     0.852173924446106     │
│      test_precision       │    0.7480915784835815     │
│        test_recall        │    0.9898989796638489     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.783,0.8497,0.7386,1.0
fog-or-smog,0.7047,0.7772,0.6373,0.9958
foggy-cityscapes,0.8283,0.8522,0.7481,0.9899


## EfficientNet B1

In [9]:
SAVE_DIR = Path('runs/classify/CNN')
MODEL_NAME = "EfficientNet_B1"
DATASET = 'fog-detection'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=f"{MODEL_NAME}_Weights.DEFAULT",
)

_transform = get_weight(model.weights).transforms()
trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=_transform,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=_transform,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=_transform))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to /home/next/.cache/torch/hub/checkpoints/efficientnet_b1-c27df63c.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 38.1MB/s]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN/EfficientNet_B1-fog-detection/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | EfficientNet     | 6.5 M  | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | train
3 | validation_metrics | MetricCollection | 0      | train
4 | test_metrics       | MetricCollectio

Epoch 9: 100%|██████████| 12/12 [00:11<00:00,  1.06it/s, v_num=1]          

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 12/12 [00:11<00:00,  1.06it/s, v_num=1]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8900862336158752     │
│          test_f1          │    0.8888888955116272     │
│      test_precision       │     0.931506872177124     │
│        test_recall        │    0.8500000238418579     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:21<00:00,  0.32it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6212121248245239     │
│          test_f1          │    0.6445497870445251     │
│      test_precision       │    0.6071428656578064     │
│        test_recall        │    0.6868686676025391     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,1.0,1.0,1.0,1.0
fog-or-smog,0.8901,0.8889,0.9315,0.85
foggy-cityscapes,0.6212,0.6445,0.6071,0.6869


In [10]:
SAVE_DIR = Path('runs/classify/CNN')
MODEL_NAME = "EfficientNet_B1"
DATASET = 'fog-or-smog'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=f"{MODEL_NAME}_Weights.DEFAULT",
)

_transform = get_weight(model.weights).transforms()
trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=_transform,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=_transform,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=_transform))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN/EfficientNet_B1-fog-or-smog/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | EfficientNet     | 6.5 M  | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | train
3 | validation_metrics | MetricCollection | 0      | train
4 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
6.5 M     Trainable params
0         Non-trainable params
6.5 M     Total params
26.063    Total estimated model params siz

Epoch 9: 100%|██████████| 52/52 [00:28<00:00,  1.83it/s, v_num=1]          

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 52/52 [00:28<00:00,  1.83it/s, v_num=1]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9762930870056152     │
│          test_f1          │    0.9766454100608826     │
│      test_precision       │    0.9956709742546082     │
│        test_recall        │    0.9583333134651184     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:23<00:00,  0.29it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5353535413742065     │
│          test_f1          │    0.6783216595649719     │
│      test_precision       │     0.51871657371521      │
│        test_recall        │    0.9797979593276978     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.8585,0.8718,0.9808,0.7846
fog-or-smog,0.9763,0.9766,0.9957,0.9583
foggy-cityscapes,0.5354,0.6783,0.5187,0.9798


In [11]:
SAVE_DIR = Path('runs/classify/CNN')
MODEL_NAME = "EfficientNet_B1"
DATASET = 'foggy-cityscapes'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=f"{MODEL_NAME}_Weights.DEFAULT",
)

_transform = get_weight(model.weights).transforms()
trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=_transform,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=_transform,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=_transform))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN/EfficientNet_B1-foggy-cityscapes/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | EfficientNet     | 6.5 M  | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | train
3 | validation_metrics | MetricCollection | 0      | train
4 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
6.5 M     Trainable params
0         Non-trainable params
6.5 M     Total params
26.063    Total estimated model param

Epoch 9: 100%|██████████| 22/22 [00:34<00:00,  0.65it/s, v_num=1]          

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 22/22 [00:34<00:00,  0.64it/s, v_num=1]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.732758641242981     │
│          test_f1          │     0.791946291923523     │
│      test_precision       │    0.6629213690757751     │
│        test_recall        │    0.9833333492279053     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:23<00:00,  0.30it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.939393937587738     │
│          test_f1          │          0.9375           │
│      test_precision       │    0.9677419066429138     │
│        test_recall        │    0.9090909361839294     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.8491,0.8889,0.8101,0.9846
fog-or-smog,0.7328,0.7919,0.6629,0.9833
foggy-cityscapes,0.9394,0.9375,0.9677,0.9091
